In [1]:
import pandas as pd
import numpy as np

In [2]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])
dfMTA = pd.read_csv('../raw_data/mta/MTA_Hourly.tdf', sep='\t',engine='python', names=['plaza', 'mtadt', 'hr', 'direction', 'vehiclesez','vehiclescash'])
dfTrans = pd.read_csv('../raw_data/soapData.tdf', sep='\t',engine='python', names = ['orderid', 'userid', 'trans', 'type', 'local','trans_dt', 'units', 'coupon', 'months', 'amt' ])

In [3]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five  

1. For each stock in 2010, return the original dataset as well as a column (“newcol”) which is the 3 daymoving average of the closing price (making sure to include the current closing price in the average).

In [12]:
out = df2010.copy()
temp = df2010.sort_values(['symb', 'retdate']).set_index(['retdate']).groupby(['symb'], as_index = False).rolling(3).agg({'cls': ['mean']}).reset_index()
temp.columns = ['symb', 'retdate', 'new_col']
pd.merge(out, temp, on = ['symb', 'retdate'], how = 'left').sort_values(['symb', 'retdate'])

,symb,retdate,opn,high,low,cls,vol,exch,new_col
491803,A,2010-01-04,22.4535,22.6252,22.2675,22.3891,2729200,NYSE,NaN
470132,A,2010-01-05,22.3247,22.3319,22.0029,22.1459,2994200,NYSE,NaN
432448,A,2010-01-06,22.1102,22.1745,22.0029,22.0672,2320200,NYSE,22.200733
466604,A,2010-01-07,22.0172,22.0458,21.8169,22.0386,2213900,NYSE,22.083900
724060,A,2010-01-08,21.9170,22.0672,21.7454,22.0315,2670800,NYSE,22.045767
...,...,...,...,...,...,...,...,...,...
169271,ZUMZ,2010-12-27,28.3000,29.1400,28.0000,28.8300,260400,NASDAQ,28.666667
222688,ZUMZ,2010-12-28,28.9100,29.2800,28.5700,28.6500,287500,NASDAQ,28.710000
237120,ZUMZ,2010-12-29,28.6900,28.9800,28.1000,28.2100,227200,NASDAQ,28.563333
142012,ZUMZ,2010-12-30,28.1000,28.7200,27.6900,27.7100,221300,NASDAQ,28.190000


2. For each stock in 2010, return the original dataset as well as a column (“newcol”) which is the 3 daymoving average of the closing price (making sure to exclude the current closing price in the average).

In [40]:
out = df2010.copy()
temp = df2010.sort_values(['symb', 'retdate']).set_index(['symb', 'retdate']).groupby(['symb']).shift(1).rolling(3).agg({'cls': ['mean']}).reset_index()
temp.columns = ['symb', 'retdate', 'new_col']
pd.merge(out, temp, on = ['symb', 'retdate'], how = 'left').sort_values(['symb', 'retdate'])


,symb,retdate,opn,high,low,cls,vol,exch,new_col
491803,A,2010-01-04,22.4535,22.6252,22.2675,22.3891,2729200,NYSE,NaN
470132,A,2010-01-05,22.3247,22.3319,22.0029,22.1459,2994200,NYSE,NaN
432448,A,2010-01-06,22.1102,22.1745,22.0029,22.0672,2320200,NYSE,NaN
466604,A,2010-01-07,22.0172,22.0458,21.8169,22.0386,2213900,NYSE,22.200733
724060,A,2010-01-08,21.9170,22.0672,21.7454,22.0315,2670800,NYSE,22.083900
...,...,...,...,...,...,...,...,...,...
169271,ZUMZ,2010-12-27,28.3000,29.1400,28.0000,28.8300,260400,NASDAQ,28.800000
222688,ZUMZ,2010-12-28,28.9100,29.2800,28.5700,28.6500,287500,NASDAQ,28.666667
237120,ZUMZ,2010-12-29,28.6900,28.9800,28.1000,28.2100,227200,NASDAQ,28.710000
142012,ZUMZ,2010-12-30,28.1000,28.7200,27.6900,27.7100,221300,NASDAQ,28.563333


3. Calculate the average correlation between closing price and volume. To do this calculate the corre-lation for each stock and then take the average over all stocks. This should return a single number.

In [57]:
temp = df2010.copy().assign(volxcls = df2010.loc[:, 'vol']*df2010.loc[:, 'cls'], 
                     cls2 = df2010.loc[:, 'cls']*df2010.loc[:, 'cls'],
                     vol2 = df2010.loc[:, 'vol']*df2010.loc[:, 'vol']).groupby(['symb'], as_index = False).agg({'retdate': ['count'],
                                                                                              'vol': ['sum'],
                                                                                              'cls': ['sum'],
                                                                                              'volxcls': ['sum'],
                                                                                              'vol2': ['sum'],
                                                                                              'cls2': ['sum']}).dropna()
temp.columns = ['symb', 'n', 'vol_sum', 'cls_sum', 'volxcls', 'vol2_sum', 'cls2_sum']
temp.loc[:, 'r'] = (temp.loc[:, 'n']*temp.loc[:, 'volxcls']\
                     - (temp.loc[:, 'vol_sum']*temp.loc[:, 'cls_sum']))/(np.sqrt(\
    (temp.loc[:, 'n']*temp.loc[:, 'vol2_sum'] - np.power(temp.loc[:, 'vol_sum'], 2))*
    (temp.loc[:, 'n']*temp.loc[:, 'cls2_sum'] - np.power(temp.loc[:, 'cls_sum'], 2))))
temp.loc[:, 'r'].mean()

/home/spnl/anaconda3/envs/databases_class_pandas/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


-0.03452222469752656

4. For each stock in 2010, calculate the percentage of the historical max closing price, up to (but notincluding) that point, that the current closing price is. Note that whenever a new max closing priceis achieved the percent would be greater than 100.

In [60]:
temp = df2010.sort_values(['symb', 'retdate']).set_index(['symb', 'retdate']).groupby('symb').shift(1).expanding().agg({'cls': ['max']}).reset_index()
temp.columns = ['symb', 'retdate', 'max_cls']
out = pd.merge(df2010, temp, on = ['symb', 'retdate'], how = 'left').sort_values(['symb', 'retdate'])
out.loc[:, 'perc'] = out.loc[:, 'cls']/out.loc[:, 'max_cls']
out.head()

,symb,retdate,opn,high,low,cls,vol,exch,max_cls,perc
491803,A,2010-01-04,22.4535,22.6252,22.2675,22.3891,2729200,NYSE,NaN,NaN
470132,A,2010-01-05,22.3247,22.3319,22.0029,22.1459,2994200,NYSE,22.3891,0.989138
432448,A,2010-01-06,22.1102,22.1745,22.0029,22.0672,2320200,NYSE,22.3891,0.985622
466604,A,2010-01-07,22.0172,22.0458,21.8169,22.0386,2213900,NYSE,22.3891,0.984345
724060,A,2010-01-08,21.9170,22.0672,21.7454,22.0315,2670800,NYSE,22.3891,0.984028


5. Using stack or unstack create a DataFrame which is one row per symbol with columns for each month in 2010. The values in those columns should be the average closing price.

In [64]:
temp = df2010.copy()
temp.loc[:, 'month'] = df2010.loc[:, 'retdate'].dt.month
out = temp.groupby(['symb', 'month']).agg({'cls': 'mean'})
out.columns = ['avgcls']
out.unstack('month')


avgcls                                                         \
month         1          2          3          4          5          6    
symb                                                                      
A      21.610195  21.551463  23.954413  25.331767  23.756085  22.257123   
AA     45.497368  40.080000  41.787391  42.481429  35.953500  33.327273   
AAME    1.406842   1.402632   1.447391   1.552857   1.630000   1.406818   
AAN    18.869121  19.637900  21.352465  22.979357  20.805500  19.341364   
AAON    5.838753   6.262458   6.446635   6.924738   7.030065   7.074482   
...          ...        ...        ...        ...        ...        ...   
ZIXI    1.974737   1.952105   2.224783   2.498095   2.358000   2.325455   
ZN      6.515789   5.847895   6.109565   5.889048   5.616500   5.167727   
ZNH    16.808947  17.476842  20.579130  24.462381  22.329500  21.950455   
ZTR    15.608421  15.256842  15.544348  15.695238  15.058000  14.881818   
ZUMZ   13.525789  13.815789  19.174783  20.576667  17.686500  16.625455   

                                                                         
month         7          8          9          10         11         12  
symb                                                                     
A      20.265348  20.179168  21.704462  24.399824  25.564062  28.201655  
AA     32.294286  32.461364  34.300000  38.287143  40.195714  43.733182  
AAME    1.259524   1.248636   1.614762   1.690952   1.837143   1.977273  
AAN    17.431429  17.361818  17.279048  17.838095  19.770952  20.375455  
AAON    7.120143   6.773468   6.872238   7.228790   7.433514   8.429082  
...          ...        ...        ...        ...        ...        ...  
ZIXI    2.251905   2.369091   2.802381   3.117143   3.861905   4.158182  
ZN      4.836667   4.890909   5.417143   5.223333   4.943810   4.787727  
ZNH    22.008571  24.139091  26.762381  33.827619  34.721905  31.009545  
ZTR    14.786667  15.081818  15.047619  15.120000  14.880000  14.303636  
ZUMZ   18.037143  16.383636  19.002857  24.678571  28.564762  29.357273  

[3362 rows x 12 columns]

# Main Problems

1. Using stack or unstack create a DataFrame which is one row per symbol with columns for eachmonth in 2010. There should be multiple columns for each month, one for the average closingprice, one for the average volume and one for the maximum volume (37 Columns: Symbol, Jan-Dec for average closing price, Jan-Dec for average volume and Jan-Dec for maximum volume). TheDataFrame returned should not have a row index.

In [66]:
temp = df2010.copy()
temp.loc[:, 'month'] = df2010.loc[:, 'retdate'].dt.month
out = temp.groupby(['symb', 'month']).agg({'cls': ['mean'],
                                           'vol': ['mean', 'max']})
out.columns = ['avgcls', 'avgvol', 'maxvol']
out.unstack('month').reset_index(drop=True)


avgcls                                                         \
month         1          2          3          4          5          6    
0      21.610195  21.551463  23.954413  25.331767  23.756085  22.257123   
1      45.497368  40.080000  41.787391  42.481429  35.953500  33.327273   
2       1.406842   1.402632   1.447391   1.552857   1.630000   1.406818   
3      18.869121  19.637900  21.352465  22.979357  20.805500  19.341364   
4       5.838753   6.262458   6.446635   6.924738   7.030065   7.074482   
...          ...        ...        ...        ...        ...        ...   
3357    1.974737   1.952105   2.224783   2.498095   2.358000   2.325455   
3358    6.515789   5.847895   6.109565   5.889048   5.616500   5.167727   
3359   16.808947  17.476842  20.579130  24.462381  22.329500  21.950455   
3360   15.608421  15.256842  15.544348  15.695238  15.058000  14.881818   
3361   13.525789  13.815789  19.174783  20.576667  17.686500  16.625455   

                                                   ...      maxvol  \
month         7          8          9          10  ...          3    
0      20.265348  20.179168  21.704462  24.399824  ...   5504400.0   
1      32.294286  32.461364  34.300000  38.287143  ...  84653400.0   
2       1.259524   1.248636   1.614762   1.690952  ...     28300.0   
3      17.431429  17.361818  17.279048  17.838095  ...    863300.0   
4       7.120143   6.773468   6.872238   7.228790  ...    162000.0   
...          ...        ...        ...        ...  ...         ...   
3357    2.251905   2.369091   2.802381   3.117143  ...   1109900.0   
3358    4.836667   4.890909   5.417143   5.223333  ...    138500.0   
3359   22.008571  24.139091  26.762381  33.827619  ...    187700.0   
3360   14.786667  15.081818  15.047619  15.120000  ...    501700.0   
3361   18.037143  16.383636  19.002857  24.678571  ...   3729600.0   

                                                                    \
month           4           5           6           7           8    
0        9147700.0  11816200.0   6002800.0   6347800.0   7305000.0   
1      100691700.0  72792800.0  63261200.0  58563000.0  33456000.0   
2          20200.0     16400.0     29400.0     34000.0     28100.0   
3        1990400.0   1936900.0   3651500.0   2253800.0    885800.0   
4         252800.0    154000.0    194200.0    107700.0     79500.0   
...            ...         ...         ...         ...         ...   
3357     1964800.0    894200.0   1546300.0    414000.0   2144200.0   
3358      654300.0    510200.0   2122000.0    182600.0    170500.0   
3359       82800.0    154700.0     61900.0     60300.0     67800.0   
3360      502100.0   2445800.0    638800.0   1140500.0    398100.0   
3361     1085400.0    676000.0   1187100.0    573800.0    405400.0   

                                                       
month          9           10          11          12  
0       6075500.0   5893000.0   7075200.0   6694200.0  
1      44203900.0  72773400.0  32993800.0  43398700.0  
2         43000.0     44700.0      8300.0     13200.0  
3       1494600.0   2959400.0   1512700.0   1213800.0  
4        128200.0     54800.0    105900.0    148400.0  
...           ...         ...         ...         ...  
3357    1480100.0   3326200.0   1862400.0   2852500.0  
3358     520500.0    196700.0    174600.0    192300.0  
3359      81200.0    271000.0    115500.0    121400.0  
3360     588700.0   5483500.0   1818100.0   1500800.0  
3361    1431000.0    860600.0   1072000.0   1819400.0  

[3362 rows x 36 columns]

2. Using stack or unstack create a DataFrame which is one row per symbol with 12 columns whichshould be the cumulative volume for that month (including that month) over the entire year of 2010.E.g. This should be the running sum, but then accumulated per month.

In [83]:
temp = df2010.copy()
temp.loc[:, 'month'] = df2010.loc[:, 'retdate'].dt.month
out = temp.groupby(['symb', 'month']).agg({'vol': 'sum'}).reset_index().set_index(['month']).groupby(['symb']).expanding().agg({'vol': ['sum']})
out.unstack('month')


vol                                                          \
                sum                                                           
month            1             2             3             4             5    
symb                                                                          
A      5.719570e+07  1.346428e+08  2.205878e+08  2.966099e+08  3.858136e+08   
AA     1.015467e+09  1.705728e+09  2.499604e+09  3.294147e+09  4.141669e+09   
AAME   9.790000e+04  2.277000e+05  3.171000e+05  4.150000e+05  4.987000e+05   
AAN    7.071800e+06  1.635490e+07  2.933390e+07  4.352050e+07  6.501920e+07   
AAON   1.618700e+06  2.914800e+06  4.492500e+06  6.097000e+06  8.007900e+06   
...             ...           ...           ...           ...           ...   
ZIXI   1.258380e+07  1.934160e+07  3.125370e+07  4.344850e+07  5.176940e+07   
ZN     1.793300e+06  3.855400e+06  5.331200e+06  9.569600e+06  1.288800e+07   
ZNH    1.123200e+06  1.638800e+06  2.679000e+06  3.521700e+06  4.731400e+06   
ZTR    5.596700e+06  1.183340e+07  2.024470e+07  2.699310e+07  3.868020e+07   
ZUMZ   6.714900e+06  9.569100e+06  2.169870e+07  3.135690e+07  3.804650e+07   

                                                                             \
                                                                              
month            6             7             8             9             10   
symb                                                                          
A      4.585995e+08  5.411047e+08  6.168390e+08  6.848937e+08  7.499596e+08   
AA     4.738995e+09  5.293511e+09  5.760258e+09  6.298012e+09  6.929098e+09   
AAME   5.802000e+05  6.736000e+05  8.109000e+05  1.020300e+06  1.124900e+06   
AAN    8.949180e+07  1.112873e+08  1.237275e+08  1.358699e+08  1.503352e+08   
AAON   9.703700e+06  1.073800e+07  1.170670e+07  1.272710e+07  1.333970e+07   
...             ...           ...           ...           ...           ...   
ZIXI   6.088810e+07  6.580950e+07  7.600710e+07  8.713110e+07  1.065471e+08   
ZN     1.692120e+07  1.859160e+07  1.953650e+07  2.291330e+07  2.436890e+07   
ZNH    5.395600e+06  5.990600e+06  6.595800e+06  7.263100e+06  8.776300e+06   
ZTR    4.522060e+07  5.616260e+07  6.200330e+07  6.902590e+07  8.638720e+07   
ZUMZ   4.437430e+07  4.974900e+07  5.560440e+07  6.186070e+07  6.801540e+07   

                                   
                                   
month            11            12  
symb                               
A      8.138079e+08  8.882545e+08  
AA     7.387581e+09  7.847254e+09  
AAME   1.196200e+06  1.238800e+06  
AAN    1.623832e+08  1.743043e+08  
AAON   1.411410e+07  1.519570e+07  
...             ...           ...  
ZIXI   1.219827e+08  1.357571e+08  
ZN     2.591780e+07  2.775080e+07  
ZNH    1.003810e+07  1.144780e+07  
ZTR    1.006989e+08  1.176372e+08  
ZUMZ   7.382950e+07  8.300600e+07  

[3362 rows x 12 columns]

3. Using stack or unstack create a DataFrame which is one row per symbol with columns for eachmonth in 2010 and 2011. The values in those columns should be the average closing price for that month.

In [90]:
temp = pd.concat([df2010, df2011]).sort_values(['symb', 'retdate']).reset_index(drop = True)
temp.loc[:, 'year-month'] = temp.loc[:, 'retdate'].dt.strftime('%Y-%m')
out = temp.groupby(['symb', 'year-month']).agg({'cls': 'mean'})
out.columns = ['avgcls']
out.unstack('year-month')

avgcls                                                         \
year-month    2010-01    2010-02    2010-03    2010-04    2010-05    2010-06   
symb                                                                           
A           21.610195  21.551463  23.954413  25.331767  23.756085  22.257123   
AA          45.497368  40.080000  41.787391  42.481429  35.953500  33.327273   
AAME         1.406842   1.402632   1.447391   1.552857   1.630000   1.406818   
AAN         18.869121  19.637900  21.352465  22.979357  20.805500  19.341364   
AAON         5.838753   6.262458   6.446635   6.924738   7.030065   7.074482   
...               ...        ...        ...        ...        ...        ...   
ZNGA              NaN        NaN        NaN        NaN        NaN        NaN   
ZNH         16.808947  17.476842  20.579130  24.462381  22.329500  21.950455   
ZTR         15.608421  15.256842  15.544348  15.695238  15.058000  14.881818   
ZUMZ        13.525789  13.815789  19.174783  20.576667  17.686500  16.625455   
ZX                NaN        NaN        NaN        NaN        NaN        NaN   

                                                        ...             \
year-month    2010-07    2010-08    2010-09    2010-10  ...    2011-03   
symb                                                    ...              
A           20.265348  20.179168  21.704462  24.399824  ...  31.515522   
AA          32.294286  32.461364  34.300000  38.287143  ...  49.644783   
AAME         1.259524   1.248636   1.614762   1.690952  ...   2.102609   
AAN         17.431429  17.361818  17.279048  17.838095  ...  23.715652   
AAON         7.120143   6.773468   6.872238   7.228790  ...   9.033174   
...               ...        ...        ...        ...  ...        ...   
ZNGA              NaN        NaN        NaN        NaN  ...        NaN   
ZNH         22.008571  24.139091  26.762381  33.827619  ...  22.581739   
ZTR         14.786667  15.081818  15.047619  15.120000  ...  13.911304   
ZUMZ        18.037143  16.383636  19.002857  24.678571  ...  25.247391   
ZX                NaN        NaN        NaN        NaN  ...        NaN   

                                                                              \
year-month    2011-04    2011-05    2011-06    2011-07    2011-08    2011-09   
symb                                                                           
A           33.624825  35.958848  34.706077  33.940270  25.146491  24.256081   
AA          51.406500  50.562857  46.260000  47.034000  37.068261  33.628571   
AAME         2.090500   2.055714   2.036364   2.050000   1.911739   1.951429   
AAN         26.443000  27.622381  26.809545  27.259500  24.804783  26.089048   
AAON         9.571560   9.633152   9.610982  10.220215   8.189374   7.271329   
...               ...        ...        ...        ...        ...        ...   
ZNGA              NaN        NaN        NaN        NaN        NaN        NaN   
ZNH         24.281000  26.915238  25.074091  30.315000  32.535652  28.711905   
ZTR         13.770000  13.780952  13.549091  13.520000  12.660870  12.472381   
ZUMZ        28.283000  28.687143  24.236364  27.115000  20.089565  18.097619   
ZX                NaN   6.013077   4.848182   5.554000   5.233913   4.486667   

                                             
year-month    2011-10    2011-11    2011-12  
symb                                         
A           24.525852  26.134271  24.983648  
AA          30.314286  30.121429  27.455714  
AAME         1.841429   1.885238   1.987619  
AAN         26.571429  25.123810  26.617143  
AAON         8.404448   9.367619   9.420114  
...               ...        ...        ...  
ZNGA              NaN        NaN   9.415000  
ZNH         26.965714  26.234762  25.829048  
ZTR         12.240000  12.280000  12.139048  
ZUMZ        21.510476  22.502381  28.425714  
ZX           4.510000   4.415238   3.261905  

[3561 rows x 24 columns]

4. For stocks in 2010, write a query which creates a dataset containing closing price, symbol, retdateand the nominal change between yesterday’s closing price and today’s opening price. Ignore holes inthe data, so that if the stock misses a day the change in price is from the last time listed.

In [96]:
df2010_cls = df2010.sort_values(['symb', 'retdate']).set_index(['symb', 'retdate']).groupby('symb').shift(1).loc[:, ['cls']].reset_index()
df2010_cls.columns = ['symb', 'retdate', 'yestcls']
out = pd.merge(df2010, df2010_cls, on = ['symb', 'retdate'], how = 'left').sort_values(['symb', 'retdate'])
out.loc[:, 'diff'] = out.loc[:, 'yestcls'] - out.loc[:, 'opn']
out.loc[:, ['cls', 'symb', 'retdate', 'diff']]

,cls,symb,retdate,diff
491803,22.3891,A,2010-01-04,NaN
470132,22.1459,A,2010-01-05,0.0644
432448,22.0672,A,2010-01-06,0.0357
466604,22.0386,A,2010-01-07,0.0500
724060,22.0315,A,2010-01-08,0.1216
...,...,...,...,...
169271,28.8300,ZUMZ,2010-12-27,0.3500
222688,28.6500,ZUMZ,2010-12-28,-0.0800
237120,28.2100,ZUMZ,2010-12-29,-0.0400
142012,27.7100,ZUMZ,2010-12-30,0.1100
